In [3]:
!apt-get update -q

zsh:1: command not found: apt-get


In [4]:
!apt-get install -y openjdk-11-jdk-headless

zsh:1: command not found: apt-get


In [5]:
from google.colab import drive
drive.mount('/content/drive')


ModuleNotFoundError: No module named 'google.colab'

**Initialize Spark**

In [6]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget https://archive.apache.org/dist/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz
!tar -xvf spark-3.5.1-bin-hadoop3.tgz
!pip install -q findspark

zsh:1: command not found: apt-get
zsh:1: command not found: wget
tar: Error opening archive: Failed to open 'spark-3.5.1-bin-hadoop3.tgz'


In [7]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.1-bin-hadoop3"

import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, regexp_extract, split, year, month, dayofmonth, median, sum as Fsum, to_date, create_map, lit, lower, element_at
from pyspark.sql.functions import udf
from pyspark.sql.types import *

Exception: Unable to find py4j in /content/spark-3.5.1-bin-hadoop3/python, your SPARK_HOME may not be configured correctly

**FULL CLEANING PIPELINE**

In [ ]:

# FINAL PySpark Cleaning Notebook for IPL Winner Prediction


# Install Spark (only for Colab)
!apt-get install -qq openjdk-11-jdk-headless > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz
!tar xf spark-3.5.1-bin-hadoop3.tgz
!pip install -q findspark

# 1) Initialize Spark
import findspark
findspark.init("/content/spark-3.5.1-bin-hadoop3")

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = SparkSession.builder \
    .appName("IPL_Winner_Cleaning_Final") \
    .getOrCreate()


# 2) Load datasets


matches_path = "/content/drive/MyDrive/ipl_match_dataset/matches.csv"
deliveries_path = "/content/drive/MyDrive/ipl_match_dataset/deliveries.csv"

matches = spark.read.csv(matches_path, header=True, inferSchema=True)
deliveries = spark.read.csv(deliveries_path, header=True, inferSchema=True)

print("Loaded matches rows:", matches.count())
print("Loaded deliveries rows:", deliveries.count())


# 3) Drop rows where 'winner' is NULL

matches = matches.dropna(subset=["winner"])


# 4) Impute missing player_of_match = "Unknown"

matches = matches.fillna({"player_of_match": "Unknown"})


# 5) Drop unwanted columns: method, umpire1, umpire2

cols_drop = ["method", "umpire1", "umpire2"]
for c in cols_drop:
    if c in matches.columns:
        matches = matches.drop(c)


# 6) Median imputation for numeric columns (result_margin, target_runs, target_overs)


def fill_median(df, colname):
    if colname in df.columns:
        try:
            med = df.approxQuantile(colname, [0.5], 0.05)[0]
            return df.fillna({colname: med})
        except:
            return df
    return df

for colname in ["result_margin", "target_runs", "target_overs"]:
    matches = fill_median(matches, colname)


# 7) Split season into season_start and season_end


if "season" in matches.columns:
    matches = matches.withColumn("season_parts", split(col("season"), "/"))
    matches = matches.withColumn("season_start", col("season_parts").getItem(0).cast("int"))
    matches = matches.withColumn(
        "season_end",
        when(
            col("season_parts").getItem(1).isNotNull() & (length(col("season_parts").getItem(1)) == 2),
            concat(lit("20"), col("season_parts").getItem(1)).cast("int")
        ).when(
            col("season_parts").getItem(1).isNotNull(),
            col("season_parts").getItem(1).cast("int")
        ).otherwise(col("season_parts").getItem(0).cast("int"))
    )
    matches = matches.drop("season_parts")


# 8) Standardize team names


# Standardize team names (lowercase already applied)
team_name_mapping = {
    'delhi daredevils': 'delhi capitals',
    'kings xi punjab': 'punjab kings',
    'rising pune supergiants': 'pune warriors',
    'rising pune supergiant': 'pune warriors',
    'gujarat lions': 'gujarat titans',
    'deccan chargers': 'sunrisers hyderabad',
    'royal challengers bengaluru': 'royal challengers bangalore'
}

# Create a flat list for Spark create_map
mapping_expr = create_map(
    [lit(key) for key_value in team_name_mapping.items() for key in key_value]
)

# Replace team names in these columns
team_cols = ["winner", "team1", "team2", "toss_winner"]

for c in team_cols:
    if c in matches.columns:
        matches = matches.withColumn(
            c,
            when(
                col(c).isin(list(team_name_mapping.keys())),
                element_at(mapping_expr, col(c))
            ).otherwise(col(c))
        )



# 9) Convert ALL string columns in both datasets to lowercase


string_cols_matches = [c for c, t in matches.dtypes if t == 'string']
for c in string_cols_matches:
    matches = matches.withColumn(c, lower(col(c)))

string_cols_deliveries = [c for c, t in deliveries.dtypes if t == 'string']
for c in string_cols_deliveries:
    deliveries = deliveries.withColumn(c, lower(col(c)))


# 10) Drop unwanted columns in deliveries (not needed for ML)


deliv_cols_drop = ["player_dismissed", "dismissal_kind", "fielder", "super_sub"]
for c in deliv_cols_drop:
    if c in deliveries.columns:
        deliveries = deliveries.drop(c)

# 11) Select ML-safe features ONLY (NO leakage columns)


leakage_cols = ["win_by_runs", "win_by_wickets", "result", "result_margin"]
for c in leakage_cols:
    if c in matches.columns:
        matches = matches.drop(c)

# Extract date features if present
if "date" in matches.columns:
    matches = matches.withColumn("date", to_date(col("date")))
    matches = matches.withColumn("year", year("date"))
    matches = matches.withColumn("month", month("date"))
    matches = matches.withColumn("day", dayofmonth("date"))
    matches = matches.drop("date")

# Keep only features known BEFORE match:
final_features = [
    "match_id", "team1", "team2", "toss_winner", "toss_decision",
    "venue", "city", "season_start", "season_end", "year", "month",
    "winner"        # label
]

final_df = matches.select([c for c in final_features if c in matches.columns])

print("Rows in final cleaned df:", final_df.count())
final_df.show(5)


# 12) Save cleaned dataset to Drive


out_path = "/content/drive/MyDrive/ipl_match_dataset/cleaned_ipl_winner_prediction.csv"
final_df.write.csv(out_path, header=True, mode="overwrite")

print("Saved cleaned dataset to:", out_path)


# END of the code, which will be input to SparkML pipeline.



Loaded matches rows: 1095
Loaded deliveries rows: 260920
Rows in final cleaned df: 1095
+--------------------+--------------------+--------------------+-------------+--------------------+----------+------------+----------+----+-----+--------------------+
|               team1|               team2|         toss_winner|toss_decision|               venue|      city|season_start|season_end|year|month|              winner|
+--------------------+--------------------+--------------------+-------------+--------------------+----------+------------+----------+----+-----+--------------------+
|royal challengers...|kolkata knight ri...|royal challengers...|        field|m chinnaswamy sta...| bangalore|        2007|      2008|2008|    4|kolkata knight ri...|
|     kings xi punjab| chennai super kings| chennai super kings|          bat|punjab cricket as...|chandigarh|        2007|      2008|2008|    4| chennai super kings|
|    delhi daredevils|    rajasthan royals|    rajasthan royals|          bat